Cel na dziś:
(*) $||b - Ax||_2 -> min!$
LSQR - metoda najmniejszych kwadratów
CGNR - metoda najmniejszych kwadratów z regularyzacją

$A^TAx = A^Tb$

(**) $||b-Py||_2 -> min!$

$x = Py$, $P$ - niepełny rozkład Choleskiego macierzy $A^T A$


1) wygenerować rozrzedzoną macierz A: $M\times N$
2) rozwiązać (*) metodą CGNR, LSQR
3) rozwiązać (**) metodą j.w.
    dla $P$ - niepełny rozkład Choleskiego macierzy $A^TA$
4) porównać szybkość zbieżności

In [557]:
import scipy.sparse as sp
import numpy as np

N = 7
M = 10
density = 0.5

A = sp.rand(M, N, density=density)

b = np.random.rand(M)

In [558]:
# A.todense(), b

In [559]:
from scipy.sparse.linalg import cg, LinearOperator, lsqr
from numpy.linalg import lstsq
from numpy.linalg import norm

def atax(x):
    return A.T @ (A @ x)

# Run cg with LinearOperator on `normal`
x_cg, _ = cg(LinearOperator(shape=(N, N), matvec=atax), A.T @ b)

# Run lsqr with LinearOperator on `normal`
x_lsqr = lsqr(A, b)[0]

# Run lstsq
x_lstsq = lstsq(A.todense(), b, rcond=None)[0]

norm(x_lsqr - x_cg), norm(x_lsqr - x_lstsq), norm(x_cg - x_lstsq)
# x_cg, x_lsqr, x_lstsq

(1.5176226162857613e-10, 1.2752206030012304e-10, 2.4240202540507337e-11)

3a) Wyznaczyć $P$

$P^{T} A^{T} A P x = P^{T} A^{T} b$

3b) $C_L$ = LinearOperator(....)
matvec: $P^{T} (A^{T} (A (P x)))$

3c) x = cg(C_{L}, ...)[0]

In [560]:
from scipy.sparse.linalg import spilu, spsolve_triangular
from scipy.sparse import diags

def chol(A):
    res = spilu(A)
    return res.L, res.U

L, U = chol(A.T @ A)
D = diags(np.sqrt(U.diagonal()))
L_dash = L @ D

def latalx(x):
    t = spsolve_triangular(L_dash.T, x, lower=False)
    t = A @ t
    t = A.T @ t
    t = spsolve_triangular(L_dash, t, lower=True)
    return t

def latb():
    t = A.T @ b
    t = spsolve_triangular(L_dash, t, lower=True)
    return t

x_cg_uber_fajne, _ = cg(LinearOperator(shape=(N, N), matvec=latalx), latb())

norm(x_cg_uber_fajne - x_cg), norm(x_cg_uber_fajne - x_lsqr), norm(x_cg_uber_fajne - x_lstsq)

(1.3514941186233203, 1.3514941186939367, 1.3514941186345972)